In [9]:
import sys
import importlib as imp
if ('Jupytils' in sys.modules):
    reloaded = imp.reload(Jupytils)
else:
    import Jupytils
    

<IPython.core.display.Javascript object>

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [26]:
fileName="../data/housing.csv"  # Data is located in analysis1.xlsx
dfOriginal = LoadDataSet(fileName,  columns=None, headers=-1, checkForDateTime=False);
dfL = dfOriginal
ds = dfL.values
X=ds[:,0:13]
y=ds[:,13]
dfL

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.006,18.000,2.310,0,0.538,6.575,65.200,4.090,1,296.000,15.300,396.900,4.980,24.000
1,0.027,0.000,7.070,0,0.469,6.421,78.900,4.967,2,242.000,17.800,396.900,9.140,21.600
2,0.027,0.000,7.070,0,0.469,7.185,61.100,4.967,2,242.000,17.800,392.830,4.030,34.700
3,0.032,0.000,2.180,0,0.458,6.998,45.800,6.062,3,222.000,18.700,394.630,2.940,33.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,0.045,0.000,11.930,0,0.573,6.120,76.700,2.288,1,273.000,21.000,396.900,9.080,20.600
503,0.061,0.000,11.930,0,0.573,6.976,91.000,2.167,1,273.000,21.000,396.900,5.640,23.900
504,0.110,0.000,11.930,0,0.573,6.794,89.300,2.389,1,273.000,21.000,393.450,6.480,22.000
505,0.047,0.000,11.930,0,0.573,6.030,80.800,2.505,1,273.000,21.000,396.900,7.880,11.900


In [27]:
# define base model
def baseline_model(dim=13):
    # create model
    model = Sequential()
    model.add(Dense(dim, input_dim=dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [29]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [31]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: 57.73 (42.30) MSE


In [34]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: 29.42 (27.41) MSE
